In [1]:
import sys; sys.path.append('../custom_python_packages')
from data_manipulation import data
from data_manipulation import custom_transformers as ct

In [2]:
books = data.loadAndClean('../../data/booksummaries/booksummaries.txt')
books = books[['bookGenre', 'plotSum', 'bookTitle']]

In [3]:
books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","Old Major, the old boar on the Manor Farm, ca...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","Alex, a teenager living in near-future Englan...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...",The text of The Plague is divided into five p...,The Plague


In [4]:
tokenizer = ct.Tokenizer()
tokenized_books = tokenizer.transform(books)
tokenized_books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[Old, Major, ,, the, old, boar, on, the, Manor...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[Alex, ,, a, teenager, living, in, near-future...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[The, text, of, The, Plague, is, divided, into...",The Plague


In [5]:
books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","Old Major, the old boar on the Manor Farm, ca...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","Alex, a teenager living in near-future Englan...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...",The text of The Plague is divided into five p...,The Plague


In [6]:
filterer_of_sentences_noFiltNames = ct.Filter_sentence()
filterer_of_sentences_filtNames = ct.Filter_sentence(filter_names=True)

tokened_and_filtered_summaries_names = filterer_of_sentences_noFiltNames.transform(tokenized_books)
tokened_and_filtered_summaries_NoNames = filterer_of_sentences_filtNames.transform(tokenized_books)

In [7]:
tokened_and_filtered_summaries_names.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[old, major, old, boar, manor, farm, calls, an...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[alex, teenager, living, nearfuture, england, ...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[text, plague, divided, five, parts, town, ora...",The Plague


In [8]:
tokened_and_filtered_summaries_NoNames.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[old, old, boar, manor, farm, calls, animals, ...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[teenager, living, nearfuture, england, leads,...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[text, plague, divided, five, parts, town, ora...",The Plague


In [9]:
porter = ct.StemmingLemming(prune_type="Porter")
portered_books = porter.transform(tokened_and_filtered_summaries_NoNames)

In [10]:
portered_books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[old, old, boar, manor, farm, call, anim, farm...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[teenag, live, nearfutur, england, lead, gang,...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[text, plagu, divid, five, part, town, oran, t...",The Plague


In [11]:
tokened_and_filtered_summaries_NoNames.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[old, old, boar, manor, farm, calls, animals, ...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[teenager, living, nearfuture, england, leads,...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[text, plague, divided, five, parts, town, ora...",The Plague


In [12]:
lancaster = ct.StemmingLemming(prune_type="Lancaster")
lancasted_books = lancaster.transform(tokened_and_filtered_summaries_NoNames)

In [13]:
lancasted_books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[old, old, boar, man, farm, cal, anim, farm, m...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[teen, liv, nearfut, england, lead, gang, nigh...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[text, plagu, divid, fiv, part, town, or, thou...",The Plague


In [14]:
lemmer = ct.StemmingLemming(prune_type="Lemmatization")
lemmatized_books = lemmer.transform(tokened_and_filtered_summaries_NoNames)

In [15]:
lemmatized_books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","[old, old, boar, manor, farm, call, animal, fa...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","[teenager, living, nearfuture, england, lead, ...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...","[text, plague, divide, five, part, town, oran,...",The Plague


In [16]:
books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...","Old Major, the old boar on the Manor Farm, ca...",Animal Farm
1,"[science_fiction, novella, speculative_fiction...","Alex, a teenager living in near-future Englan...",A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...",The text of The Plague is divided into five p...,The Plague


In [17]:
joiner_transformer = ct.Joiner()
joined_books = joiner_transformer.transform(lemmatized_books)
joined_books.head(3)

,bookGenre,plotSum,bookTitle
0,"[roman_à_clef, satire, childrens_literature, s...",old old boar manor farm call animal farm meeti...,Animal Farm
1,"[science_fiction, novella, speculative_fiction...",teenager living nearfuture england lead gang n...,A Clockwork Orange
2,"[existentialism, fiction, absurdist_fiction, n...",text plague divide five part town oran thousan...,The Plague
